In [27]:
from elasticsearch import Elasticsearch

In [28]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "DU6CcWicaYxZuMJTHvB+"),
    verify_certs=False
)
es.ping()

d:\learn\Elasticsearch\venv\Lib\site-packages\elasticsearch\_sync\client\__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
d:\learn\Elasticsearch\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


True

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
df = pd.read_csv("vnexpress.csv")

In [31]:
df.columns

Index(['Unnamed: 0', 'id', 'title', 'author', 'updatetime', 'wordcount',
       'publication', 'tags', 'content', 'category'],
      dtype='object')

In [32]:
df = df.drop(columns=['Unnamed: 0', 'id', 'author', 'updatetime', 'wordcount', 'publication', 'tags'])
df.head()

,title,content,category
0,Hai thứ cần kiểm tra ngay khi nhận phòng khách...,"Nếu chăn, ga, gối và tấm trải giường nhàu nhĩ...",du-lich
1,Thảo Cầm Viên nhộn nhịp ngày đầu mở cửa trở lại,"Sáng 5/11, người dân xếp hàng ở quầy chờ mua ...",du-lich
2,Cúc Phương là vườn quốc gia hàng đầu châu Á 2021,"Để giành danh hiệu trên, Cúc Phương đã vượt q...",du-lich
3,Cuộc sống không thể thiếu chocolate ở Bỉ,"""Người Bỉ khó có thể tồn tại mà không có choc...",du-lich
4,Bánh canh hẹ Phú Yên ở Sài Gòn,Du khách đến Phú Yên có thể dễ dàng thưởng th...,du-lich


In [33]:
df = df.dropna(subset=['title', 'content'])

In [34]:
import re
from pyvi import ViTokenizer, ViUtils


In [35]:
def clean_text(text):
    text = re.sub(r'[\\\'\"“”’‘]', '', text)  
    text = re.sub(r'[_]', ' ', text)  
    text = re.sub(r'[.,\/#!$%\^&\*;:{}=\_`~()”“\[\]]', '', text)  
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()  
    return text

In [36]:
with open('./vietnamese-stopwords.txt',encoding='utf-8') as f:
    stopwords = []
    for line in f:
        stopwords.append("_".join(line.strip().split()))

In [37]:
def normalize_text(text):
    text = text.lower()
    # text = ViUtils.remove_accents(text) 
    return text

def tokenize_text(text):
    return ViTokenizer.tokenize(text)

def remove_stop_words(text):
    words = text.split()
    words = [word for word in words if word not in stopwords]
    return ' '.join(words)

In [38]:
def preprocess_text(text):
    text = clean_text(text)
    text = normalize_text(text)
    text = tokenize_text(text)
    text = remove_stop_words(text)
    return text

In [39]:
df['title'] = df['title'].apply(preprocess_text)
df['content'] = df['content'].apply(preprocess_text)

In [40]:
df

,title,content,category
0,hai kiểm_tra phòng khách_sạn,chăn ga gối trải giường nhàu nhĩ phẳng_phiu ph...,du-lich
1,thảo cầm_viên nhộn_nhịp đầu mở_cửa trở_lại,511 dân xếp_hàng quầy chờ mua vé thảo cầm_viên...,du-lich
2,cúc phương vườn quốc_gia hàng_đầu châu 2021,giành danh_hiệu cúc phương đối_thủ nặng_ký chi...,du-lich
3,cuộc_sống chocolate bỉ,bỉ tồn_tại chocolate sản_xuất chocolate nổi_ti...,du-lich
4,bánh_canh hẹ phú yên sài_gòn,du_khách phú yên dễ_dàng thưởng_thức bánh_canh...,du-lich
...,...,...,...
473,tp hcm lập 3 cơ_sở điều_trị f0 khu công_nghiệp,chiều 311 lê bích loan phó_ban quản_lý khu côn...,thoi-su
474,sửa quy_định miễn_nhiệm từ_chức,phát_biểu hội_nghị tập_huấn báo_chí xây_dựng đ...,thoi-su
475,lao_động mét làm_việc,quy_định phương_án phòng_chống covid - 19 cơ_s...,thoi-su
476,xảy lợi_ích xây_dựng pháp_luật,311 chủ_tịch quốc_hội vương đình huệ thường_tr...,thoi-su


In [41]:
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
import torch

In [42]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

d:\learn\Elasticsearch\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [43]:
def encode_text(text):
    embeddings = model.encode(text, convert_to_tensor=True) 
    return embeddings


In [44]:
df['embedding'] = df['content'].apply(encode_text)

In [45]:
df.head(20)

,title,content,category,embedding
0,hai kiểm_tra phòng khách_sạn,chăn ga gối trải giường nhàu nhĩ phẳng_phiu ph...,du-lich,"[tensor(0.0552), tensor(0.0868), tensor(-0.006..."
1,thảo cầm_viên nhộn_nhịp đầu mở_cửa trở_lại,511 dân xếp_hàng quầy chờ mua vé thảo cầm_viên...,du-lich,"[tensor(0.0395), tensor(0.0360), tensor(0.0310..."
2,cúc phương vườn quốc_gia hàng_đầu châu 2021,giành danh_hiệu cúc phương đối_thủ nặng_ký chi...,du-lich,"[tensor(0.0269), tensor(0.0464), tensor(0.0096..."
3,cuộc_sống chocolate bỉ,bỉ tồn_tại chocolate sản_xuất chocolate nổi_ti...,du-lich,"[tensor(0.0164), tensor(0.0209), tensor(0.0551..."
4,bánh_canh hẹ phú yên sài_gòn,du_khách phú yên dễ_dàng thưởng_thức bánh_canh...,du-lich,"[tensor(0.0416), tensor(0.0846), tensor(0.0217..."
5,tp hcm đón quốc_tế,tp hcm dự_thảo đề_án tổ_chức đón du_lịch quốc_...,du-lich,"[tensor(0.0430), tensor(0.0257), tensor(-0.001..."
6,đam_mê mùa đông_sa_pa,nguyễn văn thi 33 hà_nội du_lịch tỉnh miền bắc...,du-lich,"[tensor(0.0460), tensor(0.0567), tensor(0.0345..."
7,lưu_ý du_lịch đà lạt giãn,đi du_lịch đà lạt đợt du_khách 5k giấy_tờ cần_...,du-lich,"[tensor(0.0107), tensor(0.0298), tensor(0.0424..."
8,8 món vội thái_lan,nhà_hàng thất_bại ấn_tượng anh_minh thaiger,du-lich,"[tensor(0.0264), tensor(0.0344), tensor(0.0270..."
9,4 quán cà_phê chụp ảnh đẹp sài_gòn,non coffee tea địa_chỉ 1175 trường_chinh phườn...,du-lich,"[tensor(0.0208), tensor(0.0506), tensor(0.0021..."


In [46]:
index_name = 'articles'
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body={
        "mappings": {
            "properties": {
                "title": {"type": "text"},
                "content": {"type": "text"},
                "category": {"type": "keyword"},
                "embedding": {"type": "dense_vector", "dims": 512}  
            }
        }
    })


d:\learn\Elasticsearch\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\learn\Elasticsearch\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [47]:
for _, row in df.iterrows():
    document = {
        "title": row['title'],
        "content": row['content'],
        "category": row['category'],
        "embedding": row['embedding'].tolist() 
    }
    es.index(index=index_name, document=document)


d:\learn\Elasticsearch\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\learn\Elasticsearch\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\learn\Elasticsearch\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\learn\Elasticsearch\venv\Lib\site-packages\urllib3\connectionpoo

In [48]:
def search_bm25(query):
    bm25_query = {
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title", "content"]
            }
        }
    }
    bm25_results = es.search(index=index_name, body=bm25_query)
    return bm25_results


In [49]:

def search_embedding(query_embedding):
    embedding_query = {
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {"query_vector": query_embedding}
                }
            }
        }
    }
    embedding_results = es.search(index=index_name, body=embedding_query)
    return embedding_results


In [50]:
def search(query):
    query_embedding = encode_text(query).tolist()

    bm25_results = search_bm25(query)
    
    embedding_results = search_embedding(query_embedding)

    combined_results = bm25_results['hits']['hits'] + embedding_results['hits']['hits']
    combined_results = sorted(combined_results, key=lambda x: x['_score'], reverse=True)

    return combined_results


In [51]:
query = "đà nẵng"
results = search(query)

for result in results:
    print(f"Title: {result['_source']['title']}, Score: {result['_score']}")

d:\learn\Elasticsearch\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Title: độc_giả giải ý_tưởng du_lịch đà_nẵng, Score: 11.47403
Title: hỗn_chiến trụ_sở công_an, Score: 6.05655
Title: lưu_ý du_lịch đà lạt giãn, Score: 5.9687934
Title: lừa khôi_phục tài_khoản facebook giá 11 tỷ đồng, Score: 4.0891047
Title: chứng_khoán đỉnh, Score: 3.541133
Title: cổ_phiếu hoà_bình dậy sóng, Score: 3.541133
Title: cổ_phiếu ngân_hàng người_hùng thành tội_đồ, Score: 3.438729
Title: dã quỳ nở vàng ngoại_thành đà_lạt, Score: 3.3702815
Title: tương_lai chiến_lược covid trung_quốc, Score: 3.2773914
Title: vietravel tung loạt sản_phẩm du_lịch miền tây, Score: 3.1642172
Title: thành_phố gọi venice phương bắc ?, Score: 1.8211983
Title: đi 11 ?, Score: 1.8211983
Title: đặc_sản hà_giang mua làm_quà, Score: 1.7695076
Title: êkíp james bond 25 cảnh hành_động, Score: 1.7081485
Title: biệt_thự trăm vanessa hudgens, Score: 1.7081485
Title: cổ_vật việt đấu_giá nước_ngoài, Score: 1.6931144
Title: phong_độ 53 lý hải, Score: 1.6461928
Title: loạt smartphone giảm_giá đầu 11, Score: 1.644881

d:\learn\Elasticsearch\venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [52]:
model.save('model.h5')